In [2]:
!pip install python-dotenv
!pip install huggingface_hub

In [3]:
from huggingface_hub import login

login()

In [4]:
# # If HF token is stored in .env:
# import os
# os.environ["HF_TOKEN"] = os.getenv('HF_TOKEN')

## Load the Model and Tokenizer from Local

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # Specify the path to your mer ged model directory
# merged_model_path = "merged_model"

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(merged_model_path)

# # Load the merged model
# model = AutoModelForCausalLM.from_pretrained(
#     merged_model_path,
#     torch_dtype=torch.float16,  # Adjust based on your saved model's dtype
#     device_map="auto"  # Automatically maps model to available devices
# )

# # If you have a GPU, ensure the model is on the GPU
# if torch.cuda.is_available():
#     model = model.cuda()

## Load the Model and Tokenizer from HF (after pushing)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

username = "jxdemesa"
model_id = f"{username}/gemma-2b-it-finetuned-medical-qa"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # Adjust based on your saved model's dtype
    device_map="auto"  # Automatically maps model to available devices
)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

## Test the finetuned model

In [ ]:
# Helper functions to get the responses in proper format

def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  <end_of_turn>\n<start_of_turn>model


  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return decoded

In [ ]:
query = """\n\n Please answer with one of the option in the bracket. Write reasoning in between <analysis></analysis>.
           Write answer in between <answer></answer>. here are the inputs Q:An 8-year-old boy is brought to the pediatrician
           by his mother with nausea, vomiting, and decreased frequency of urination. He has acute lymphoblastic leukemia
           for which he received the 1st dose of chemotherapy 5 days ago. His leukocyte count was 60,000/mm3 before starting
           chemotherapy. The vital signs include: pulse 110/min, temperature 37.0°C (98.6°F), and blood pressure 100/70 mm Hg.
           The physical examination shows bilateral pedal edema. Which of the following serum studies and urinalysis findings
           will be helpful in confirming the diagnosis of this condition? ? \n{'A': 'Hyperkalemia, hyperphosphatemia, hypocalcemia,
           and extremely elevated creatine kinase (MM)', 'B': 'Hyperkalemia, hyperphosphatemia, hypocalcemia, hyperuricemia, urine
           supernatant pink, and positive for heme', 'C': 'Hyperuricemia, hyperkalemia, hyperphosphatemia, lactic acidosis, an
            urate crystals in the urine', 'D': 'Hyperuricemia, hyperkalemia, hyperphosphatemia, and urinary monoclonal spike',
            'E': 'Hyperuricemia, hyperkalemia, hyperphosphatemia, lactic acidosis, and oxalate crystals'}"""

result = get_completion(query=query, model=model, tokenizer=tokenizer)
print(result)


  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  

 Please answer with one of the option in the bracket. Write reasoning in between <analysis></analysis>.
           Write answer in between <answer></answer>. here are the inputs Q:An 8-year-old boy is brought to the pediatrician
           by his mother with nausea, vomiting, and decreased frequency of urination. He has acute lymphoblastic leukemia
           for which he received the 1st dose of chemotherapy 5 days ago. His leukocyte count was 60,000/mm3 before starting
           chemotherapy. The vital signs include: pulse 110/min, temperature 37.0°C (98.6°F), and blood pressure 100/70 mm Hg.
           The physical examination shows bilateral pedal edema. Which of the following serum studies and urinalysis findings
           will be helpful in confirming the diagnosis of this condition? ? 
{'A': 'Hyperkalemia, hyperphosphatemia, hypocalcemia,
           and extre

In [ ]:
print(f"{result.split('Analysis:')[-1]}")



 This is a question about differential diagnosis of a 6 year old boy with acute lymphoblastic leukemia after receiving a 1st dose of chemotherapy and exhibiting some features of acute lymphoblastic leukemia. 

 From the description, the features presented include acute onset leukemia, bilateral pedal edema, high leukocyte count (60,000/mm), and hyperkalemia. The question asks which of the following laboratory findings and findings on urinalysis would be most helpful in confirming the diagnosis.

Of the answer choices, hypocalcemia, hyperuricemia, and urine protein: dipstick or urinalysis are most helpful findings to consider in differentiating between acute lymphoblastic leukemia and other causes of acute hemolytic anemia. Hypocalcemia would be most concerning as acute lymphoblastic leukemia may be a cause of severe hypocalcemia.

Choice E refers to an acute hemolytic disorder, not acute lymphoblastic leukemia. Therefore, that choice does not help answer this question.

Choice A incl